<a href="https://colab.research.google.com/github/tonystz/gitpod/blob/main/strings_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [93]:
import numpy as np
ttt = np.asarray([ "stuff" + str(i)  for i in range(0,2) ])

print( ttt.dtype, type(ttt[0]) ) 
print(ttt)

s= np.array(['abcdf'], dtype=object)
print( s.dtype, type(s[0]) )

#print("ctype:",np.ctypeslib.as_ctypes_type(s.dtype))

<U6 <class 'numpy.str_'>
['stuff0' 'stuff1']
object <class 'str'>


## all string size must equal?

In [111]:
%%writefile s.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <stdio.h>

    __global__ void say_hi(char*out_gpu, unsigned int *s_offset_gpu)
    { 
      int idx = threadIdx.x + blockIdx.x * blockDim.x;
      //printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
      //printf("thread id: [%d]  offset:%d \\n",idx,s_offset_gpu[idx]);
      printf("thread id: [%d]  offset:%d  get str:%s\\n",idx,s_offset_gpu[idx],out_gpu+s_offset_gpu[idx]);
    }
    """)

say_hi = mod.get_function("say_hi")


s= np.array(['abcm','edfg','s12w','h45q'], dtype=np.string_)

offset=[0]
for i in s:
  offset.append(offset[-1]+len(i))
s_offset=np.array(offset,dtype=np.uint32)
print('shape:',s.shape)
print(s,s.data)
print('offset:',s_offset,s_offset.shape)

s_gpu = gpuarray.to_gpu(s)
s_offset_gpu = gpuarray.to_gpu(s_offset)

say_hi(s_gpu,s_offset_gpu,block=(s.size,1,1),grid=(1,1,1))
print('modify:',s_gpu.get(),s_offset_gpu.get())

print("s_offset ctype:",np.ctypeslib.as_ctypes_type(s_offset.dtype))

Overwriting s.py


In [112]:
!python s.py

shape: (4,)
[b'abcm' b'edfg' b's12w' b'h45q'] <memory at 0x7f35263b0580>
offset: [ 0  4  8 12 16] (5,)
thread id: [0]  offset:0  get str:abcmedfgs12wh45q
thread id: [1]  offset:4  get str:edfgs12wh45q
thread id: [2]  offset:8  get str:s12wh45q
thread id: [3]  offset:12  get str:h45q
modify: [b'abcm' b'edfg' b's12w' b'h45q'] [ 0  4  8 12 16]
s_offset ctype: <class 'ctypes.c_uint'>


## test un-equal string process?

In [85]:
%%writefile t389.cu
#include <stdio.h>
#include <string.h>

#define nTPB 256

__global__ void kern_1D(char *data, unsigned *indices, unsigned num_strings){

  int idx = threadIdx.x+blockDim.x*blockIdx.x;
  if (idx < num_strings)
    printf("Hello from thread %d, offset=%d, my string is %s\n", idx, indices[idx],data+indices[idx]);
}

__global__ void kern_2D(char **data, unsigned num_strings){

  int idx = threadIdx.x+blockDim.x*blockIdx.x;
  if (idx < num_strings)
    printf("Hello from thread %d, my string is %s\n", idx, data[idx]);
}

int main(){

  const int num_strings = 3;
  const char s0[] = "s1\0";
  const char s1[] = "s2\0";
  const char s2[] = "sstz3";
  int ds[num_strings];
  ds[0] = sizeof(s0)/sizeof(char);
  ds[1] = sizeof(s1)/sizeof(char);
  ds[2] = sizeof(s2)/sizeof(char);
  // pretend we have a dynamically allocated char** array
  char **data;
  data = (char **)malloc(num_strings*sizeof(char *));
  data[0] = (char *)malloc(ds[0]*sizeof(char));
  data[1] = (char *)malloc(ds[1]*sizeof(char));
  data[2] = (char *)malloc(ds[2]*sizeof(char));
  // initialize said array
  strcpy(data[0], s0);
  strcpy(data[1], s1);
  strcpy(data[2], s2);
  // method 1: "flattening"
  char *fdata = (char *)malloc((ds[0]+ds[1]+ds[2])*sizeof(char));
  unsigned *ind   = (unsigned *)malloc(num_strings*sizeof(unsigned));
  unsigned next = 0;
  for (int i = 0; i < num_strings; i++){
    strcpy(fdata+next, data[i]);
    ind[i] = next;
    next += ds[i];}
  //copy to device
  char *d_fdata;
  unsigned *d_ind;
  cudaMalloc(&d_fdata, next*sizeof(char));
  cudaMalloc(&d_ind, num_strings*sizeof(unsigned));
  cudaMemcpy(d_fdata, fdata, next*sizeof(char), cudaMemcpyHostToDevice);
  cudaMemcpy(d_ind, ind, num_strings*sizeof(unsigned), cudaMemcpyHostToDevice);
  printf("method 1: %d  --> %d\n",(num_strings+nTPB-1)/nTPB,nTPB);
  kern_1D<<<(num_strings+nTPB-1)/nTPB, nTPB>>>(d_fdata, d_ind, num_strings);
  cudaDeviceSynchronize();
  //method 2: "2D" (pointer-to-pointer) array
  char **d_data;
  cudaMalloc(&d_data, num_strings*sizeof(char *));
  char **d_temp_data;
  d_temp_data = (char **)malloc(num_strings*sizeof(char *));
  for (int i = 0; i < num_strings; i++){
    cudaMalloc(&(d_temp_data[i]), ds[i]*sizeof(char));
    cudaMemcpy(d_temp_data[i], data[i], ds[i]*sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_data+i, &(d_temp_data[i]), sizeof(char *), cudaMemcpyHostToDevice);}
  printf("method 2:\n");
  kern_2D<<<(num_strings+nTPB-1)/nTPB, nTPB>>>(d_data, num_strings);
  cudaDeviceSynchronize();
  // method 3: managed allocations
  // start over with a managed char** array
  char **m_data;
  cudaMallocManaged(&m_data, num_strings*sizeof(char *));
  cudaMallocManaged(&(m_data[0]), ds[0]*sizeof(char));
  cudaMallocManaged(&(m_data[1]), ds[1]*sizeof(char));
  cudaMallocManaged(&(m_data[2]), ds[2]*sizeof(char));
  // initialize said array
  strcpy(m_data[0], s0);
  strcpy(m_data[1], s1);
  strcpy(m_data[2], s2);
  // call kernel directly on managed data
  printf("method 3:\n");
  kern_2D<<<(num_strings+nTPB-1)/nTPB, nTPB>>>(m_data, num_strings);
  cudaDeviceSynchronize();

  return 0;
}

Writing t389.cu


In [86]:
!nvcc -o t389 t389.cu
!compute-sanitizer ./t389

========= COMPUTE-SANITIZER
method 1: 1  --> 256
Hello from thread 0, offset=0, my string is s1
Hello from thread 1, offset=4, my string is s2
Hello from thread 2, offset=8, my string is sstz3
method 2:
Hello from thread 0, my string is s1
Hello from thread 1, my string is s2
Hello from thread 2, my string is sstz3
method 3:
Hello from thread 0, my string is s1
Hello from thread 1, my string is s2
Hello from thread 2, my string is sstz3
========= ERROR SUMMARY: 0 errors


In [77]:
%%writefile s2.py
import time
import numpy as np
from pycuda import driver, compiler, gpuarray, tools
import math
from sys import getsizeof

import pycuda.autoinit

kernel_code1 = """
__global__ void test1(char** d_wordList) {
    (d_wordList[blockIdx.x][threadIdx.x])++;      
}
    """

kernel_code2 = """
__global__ void test2(char* d_wordList, size_t *offsets) {

    int idx = threadIdx.x+blockDim.x*blockIdx.x;
    printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
    printf("Hello from thread %d, my string is %s ,offsets:%u \\n", idx, d_wordList+offsets[idx],offsets[blockIdx.x]);
    (d_wordList[offsets[blockIdx.x] + threadIdx.x])++;
}
    """




mod = compiler.SourceModule(kernel_code1)
ker_test1 = mod.get_function("test1")



wordList = ['asd','bsd','csd']

d_words = []

for word in wordList:
    d_words.append(gpuarray.to_gpu(np.array(word, dtype=str)))

d_wordList = gpuarray.to_gpu(np.array([word.ptr for word in d_words], dtype=np.uintp))

ker_test1(d_wordList, block=(3,1,1), grid=(3,1,1))

for word in d_words:
  result = word.get()
  print(result)

mod2 = compiler.SourceModule(kernel_code2)
ker_test2 = mod2.get_function("test2")

d_words2 = gpuarray.to_gpu(np.array(['asd','bsd','csd'], dtype=np.string_))
offsets = gpuarray.to_gpu(np.array([0,3,6,9], dtype=np.uint64))
ker_test2(d_words2, offsets, block=(3,1,1), grid=(1,1,1))
h_words2 = d_words2.get()
print(h_words2)


Overwriting s2.py


In [78]:
!python s2.py

𐅢sd
𐅣sd
𐅤sd
index: threadIdx.x=0 blockIdx.x=0  blockDim.x=3 
index: threadIdx.x=1 blockIdx.x=0  blockDim.x=3 
index: threadIdx.x=2 blockIdx.x=0  blockDim.x=3 
Hello from thread 0, my string is asdbsdcsd ,offsets:0 
Hello from thread 1, my string is bsdcsd ,offsets:0 
Hello from thread 2, my string is csd ,offsets:0 
[b'bte' b'bsd' b'csd']


### reference:
https://stackoverflow.com/questions/34700999/cuda-passing-char-to-kernel/34712905